In [ ]:
import psycopg2
import redis

In [ ]:
# Configuración de PostgreSQL
pg_conn = psycopg2.connect(
    dbname='Prueba',
    user='Usuario_Prueba',
    password='Password_Prueba',
    host='localhost',
    port='5432'
)
pg_cursor = pg_conn.cursor()

# Configuración de Redis
redis_client = redis.StrictRedis(
    host='localhost',
    port=6379,
    db=0,
    decode_responses=True  # Para decodificar las respuestas a strings
)

In [ ]:
def get_data(key):
    data = redis_client.get(key)
    if data:
        print(f"Datos obtenidos de la caché: {data}")
        return data
    
    pg_cursor.execute('SELECT nombre, descripcion FROM insercion WHERE id = %s', (key,))
    result = pg_cursor.fetchone()
    if result:
        data = {'nombre': result[0], 'descripcion': result[1]}
        redis_client.setex(key, 3600, str(data))
        print(f"Datos obtenidos de la base de datos y almacenados en caché: {data}")
        return data
    else:
        print("No se encontraron datos")
        return None


In [ ]:
def save_data(key, nombre, descripcion):
    try:
        pg_cursor.execute('BEGIN')
        pg_cursor.execute('INSERT INTO insercion (id, nombre, descripcion) VALUES (%s, %s, %s) ON CONFLICT (id) DO UPDATE SET nombre = EXCLUDED.nombre, descripcion = EXCLUDED.descripcion', (key, nombre, descripcion))
        pg_conn.commit()
        data = {'nombre': nombre, 'descripcion': descripcion}
        redis_client.setex(key, 3600, str(data))
        print(f"Datos guardados en PostgreSQL y en caché: {data}")
    except Exception as e:
        pg_conn.rollback()
        print(f"Error al guardar los datos: {e}")
        raise e


In [ ]:
def delete_data(key):
    try:
        pg_cursor.execute('BEGIN')
        pg_cursor.execute('DELETE FROM insercion WHERE id = %s', (key,))
        pg_conn.commit()
        redis_client.delete(key)
        print(f"Datos eliminados de PostgreSQL y caché para la clave: {key}")
    except Exception as e:
        pg_conn.rollback()
        print(f"Error al eliminar los datos: {e}")
        raise e

In [ ]:
# Ejemplo de uso
save_data(1, 'Ejemplo', 'Esta es una descripción de ejemplo.')
get_data(1)
delete_data(1)

In [ ]:
# Cerrar el cursor y la conexión
pg_cursor.close()
pg_conn.close()